In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import datetime
import time

In [ ]:
def get_url(url):
    try:
        r = requests.get(url)
    except Exception as e:
        print(e)
        return None
    else:
        # do something if no exception
        #print(f"Status code:  {r.status_code}")
        #print(f"Content type: {r.headers['content-type']}")
        # print(r.text[:1000])
        return r.text

In [183]:
kadencja = 8

In [184]:
url = 'https://www.sejm.gov.pl/Sejm8.nsf/agent.xsp?symbol=posglos&NrKadencji='+str(kadencja)
bs = BeautifulSoup(get_url(url), 'lxml')

In [185]:
tab = bs.table
#dir()

In [186]:
glosowania = []
nr_pos = None

for row in tab.find_all('tr'):
    
    row_elements = row.find_all('td')
    if(len(row_elements)):
        try:
            nr_pos = int(row_elements[0].text)
            glosowania.append( [ nr_pos, row_elements[1].text,'https://www.sejm.gov.pl/Sejm8.nsf/' + row_elements[1].a.get('href'),row_elements[2].text ] )
        except:
            glosowania.append( [ nr_pos, row_elements[1].text,'https://www.sejm.gov.pl/Sejm8.nsf/' + row_elements[1].a.get('href'),row_elements[2].text ] )
            pass
        
#glosowania  

In [187]:
miesiace = ['stycznia', 'lutego', 'marca', 'kwietnia','maja', 'czerwca',
            'lipca', 'sierpnia','września', 'października', 'listopada', 'grudnia']

for row in glosowania:
    string = row[1].split(' ')
    row[1] = datetime.datetime(int(string[2]), miesiace.index(string[1]) +1 , int(string[0])  )

In [188]:
df = pd.DataFrame(glosowania, columns=['nr_pos','data','url','liczba_glos'])
df['kadencja'] = kadencja
df.to_csv('glosowania.csv',index=False)

In [189]:
df[:10]

,nr_pos,data,url,liczba_glos,kadencja
0,86,2019-09-11,https://www.sejm.gov.pl/Sejm8.nsf/agent.xsp?sy...,88,8
1,86,2019-10-16,https://www.sejm.gov.pl/Sejm8.nsf/agent.xsp?sy...,38,8
2,85,2019-07-31,https://www.sejm.gov.pl/Sejm8.nsf/agent.xsp?sy...,24,8
3,85,2019-08-09,https://www.sejm.gov.pl/Sejm8.nsf/agent.xsp?sy...,11,8
4,85,2019-08-30,https://www.sejm.gov.pl/Sejm8.nsf/agent.xsp?sy...,38,8
5,84,2019-07-17,https://www.sejm.gov.pl/Sejm8.nsf/agent.xsp?sy...,7,8
6,84,2019-07-19,https://www.sejm.gov.pl/Sejm8.nsf/agent.xsp?sy...,169,8
7,83,2019-07-03,https://www.sejm.gov.pl/Sejm8.nsf/agent.xsp?sy...,7,8
8,83,2019-07-04,https://www.sejm.gov.pl/Sejm8.nsf/agent.xsp?sy...,111,8
9,82,2019-06-12,https://www.sejm.gov.pl/Sejm8.nsf/agent.xsp?sy...,6,8


# part II

In [190]:
df = pd.read_csv('glosowania.csv')

In [191]:
def get_daily_poll(df_row):
    glosowanie_daily = []
    url_core = 'https://www.sejm.gov.pl/Sejm8.nsf/'

    url = df_row['url']
    nr_posiedzenia = df_row['nr_pos']
    data = df_row['data']
    kadencja = df_row['kadencja']
    
    bs = BeautifulSoup(get_url(url), 'lxml')
    tab = bs.table

    for row in tab.find_all('tr'):
        row_elements = row.find_all('td')
        if(len(row_elements)):
            try:
                glosowanie_daily.append([row_elements[0].text,url_core + row_elements[0].a.get('href'),
                                       row_elements[1].text, row_elements[2].text, nr_posiedzenia, data, kadencja ] )
            except Exception as e:
                print(e)
                pass
            
    df_daily = pd.DataFrame(glosowanie_daily, columns=['#_w_dniu','url','godzina','opis','nr_posiedzenia','data', 'kadencja'])
    return df_daily

In [192]:
df_main = pd.DataFrame([], columns=['#_w_dniu','url','godzina','opis','nr_posiedzenia','data', 'kadencja'])

for i in range(len(df)):
    time.sleep(1)
    a=get_daily_poll(df.loc[i])
    df_main =df_main.append( a )

In [193]:
df_main.to_csv('all_glosowania_'+str(kadencja)+'.csv',index=False)
#df_main

In [194]:
len(df_main)

8180

In [195]:
df_main.sample(10)

,#_w_dniu,url,godzina,opis,nr_posiedzenia,data,kadencja
59,60,https://www.sejm.gov.pl/Sejm8.nsf/agent.xsp?sy...,20:36:42,Pkt 6. porz. dzien. Pierwsze czytanie poselski...,15,2016-03-31,8
23,26,https://www.sejm.gov.pl/Sejm8.nsf/agent.xsp?sy...,10:28:32,Pkt 9. porz. dzien. Sprawozdanie Komisji o rzą...,36,2017-02-24,8
21,41,https://www.sejm.gov.pl/Sejm8.nsf/agent.xsp?sy...,23:03:49,Pkt 12. porz. dzien. Wybór członków Trybunału ...,1,2015-11-18,8
125,126,https://www.sejm.gov.pl/Sejm8.nsf/agent.xsp?sy...,11:21:05,Pkt 1. porz. dzien. Sprawozdanie Komisji o rzą...,76,2019-01-16,8
2,3,https://www.sejm.gov.pl/Sejm8.nsf/agent.xsp?sy...,10:25:21,68. posiedzenie Sejmu Rzeczypospolitej Polskie...,68,2018-09-12,8
51,64,https://www.sejm.gov.pl/Sejm8.nsf/agent.xsp?sy...,20:30:26,Pkt 23. porz. dzien. Sprawozdanie Komisji o rz...,69,2018-10-04,8
83,84,https://www.sejm.gov.pl/Sejm8.nsf/agent.xsp?sy...,22:18:38,Pkt 3. porz. dzien. Sprawozdanie Komisji o rzą...,86,2019-09-11,8
96,109,https://www.sejm.gov.pl/Sejm8.nsf/agent.xsp?sy...,13:06:44,Pkt 38. porz. dzien. Sprawozdanie Komisji o uc...,67,2018-07-20,8
34,37,https://www.sejm.gov.pl/Sejm8.nsf/agent.xsp?sy...,11:45:19,Pkt 13. porz. dzien. Pierwsze czytanie rządowe...,78,2019-03-15,8
92,100,https://www.sejm.gov.pl/Sejm8.nsf/agent.xsp?sy...,22:11:15,Pkt 28. porz. dzien. Sprawozdanie Komisji o po...,83,2019-07-04,8


In [ ]:
## pakiet tqdm 
## %%time

In [196]:
### laczenie ###
df = pd.read_csv('all_glosowania_7.csv')
df = df.append( pd.read_csv('all_glosowania_8.csv') )
df = df.append( pd.read_csv('all_glosowania_9.csv') )

df.to_csv('all_glosowania.csv',index=False)


In [199]:
#df.sample(10)
len(df)

14530